In [1]:
# # for discoverersdetours


In [2]:
# dictionary of datasets

df_all = {}
categories = ['inventors','nobels']
# files downloaded from wikipedia and top cleaned
categorylinks ={'inventors' : 'https://en.wikipedia.org/wiki/List_of_inventors',\
                'nobels': 'https://en.wikipedia.org/wiki/List_of_Nobel_laureates'}

In [3]:
# set category

category = 'inventors'
categorylink = categorylinks[category]



In [4]:
pklpath = 'pkls'

In [5]:
#Nobels

import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle


if category == 'nobels':
  if True:
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/List_of_Nobel_laureates"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the Nobel laureates
    table = soup.find("table", {"class": "wikitable sortable"})

    # Initialize lists to store data
    names = []
    name_links = []
    fields = []
    years = []

    # Extract field names from the header row (first row)
    header_row = table.find("tr")
    field_columns = header_row.find_all("th")[1:]  # exclude the first column ("Year")

    for column in field_columns:
        field = column.find("a").text.strip()
        fields.append(field)

    # Extract data from each subsequent row in the table
    rows = table.find_all("tr")[1:]  # skip the header row
    for row in rows:
        columns = row.find_all(["th", "td"])
        year = columns[0].text.strip()

        for i in range(1, len(columns)):
            nobelists = columns[i].find_all("span", {"class": "vcard"})

            # Extract field for each laureate
            field = fields[i - 1]  # use i - 1 to align with field columns

            for nobelist in nobelists:
                name_span = nobelist.find("span", {"class": "fn"})
                if name_span:
                    name = name_span.text.strip()
                    name_link = "https://en.wikipedia.org" + name_span.find("a").get("href")

                    names.append(name)
                    name_links.append(name_link)
                    fields.append(field)  # Append the correct field
                    years.append(year)

    # Create a DataFrame
    df = pd.DataFrame({
        "Name": names,
        "Name_Link": name_links,
        "Field": fields[:len(names)],  # Ensure fields list matches the length of names
        "Year": years
    })

    # Print the first few rows of the dataframe to verify
    print(df.head())


    # Pickle the DataFrame
    pickle.dump([category,df], open(pklpath+'/'+ category+'df.pkl', 'wb'))

  else:

    # Load the pickled DataFrame
    [category,df] = pickle.load(open(pklpath+'/'+ category+'df.pkl', 'rb'))


In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import pickle

if category == 'inventors':

    if True:
        # Step 1: Fetch the HTML content from Wikipedia link
        url = categorylink
        response = requests.get(url)
        html_content = response.text

        # Step 2: Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Step 3: Extract information starting from the first relevant heading
        data = []
        base_url = "https://en.wikipedia.org"
        current_heading = soup.find('span', {'id': 'A'}).find_parent('h3')
        
        while current_heading:
            next_sibling = current_heading.find_next_sibling()
            while next_sibling and next_sibling.name != 'h3':
                if next_sibling.name == 'ul':
                    for li in next_sibling.find_all('li'):
                        person_name_tag = li.find('a')
                        if person_name_tag:
                            person_name = re.sub(r'\s+', ' ', person_name_tag.get_text()).strip()
                            person_link = base_url + person_name_tag['href'] if 'href' in person_name_tag.attrs else None
                        else:
                            person_name = None
                            person_link = None

                        data.append([person_name, person_link])
                next_sibling = next_sibling.find_next_sibling()
            current_heading = next_sibling

        # Step 4: Create DataFrame
        df = pd.DataFrame(data, columns=['Name', 'Name_Link'])

        # Pickle the DataFrame
        pickle.dump([category, df], open(pklpath + '/' + category + 'df.pkl', 'wb'))

    else:
        # Load the pickled DataFrame
        [category, df] = pickle.load(open(pklpath + '/' + category + 'df.pkl', 'rb'))


In [19]:
print(df.shape)
df.head(-5)

(1074, 2)


,Name,Name_Link
0,Vitaly Abalakov,https://en.wikipedia.org/wiki/Vitaly_Abalakov
1,Ernst Karl Abbe,https://en.wikipedia.org/wiki/Ernst_Abbe
2,Hovannes Adamian,https://en.wikipedia.org/wiki/Hovannes_Adamian
3,Samuel W. Alderson,https://en.wikipedia.org/wiki/Samuel_W._Alderson
4,Alexandre Alexeieff,https://en.wikipedia.org/wiki/Alexandre_Alexeieff
...,...,...
1064,Konrad Zuse,https://en.wikipedia.org/wiki/Konrad_Zuse
1065,Vasily Zvyozdochkin,https://en.wikipedia.org/wiki/Vasily_Zvyozdochkin
1066,Vladimir Zworykin,https://en.wikipedia.org/wiki/Vladimir_Zworykin
1067,Creativity techniques,https://en.wikipedia.org/wiki/Creativity_techn...


In [21]:
# Clean
df.columns

df = df.dropna(subset=['Name', 'Name_Link'])

In [23]:
# main information

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


if True:

  def get_infobox_details(url):
      headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
      }
      response = requests.get(url, headers=headers)
      soup = BeautifulSoup(response.content, 'html.parser')

      # Extract infobox
      infobox = soup.find('table', class_='infobox biography vcard')

      if not infobox:
          return None, None, [], [], None

      # Initialize default values
      birthplace = None
      birthplace_link = None
      institutions = []
      institutions_links = []
      photo_link = None

      for row in infobox.find_all('tr'):
          header = row.find('th')
          if header:
              header_text = header.get_text(strip=True)
              if 'Born' in header_text:
                  birth_info = row.find('td')
                  if birth_info:
                      birthplace_div = birth_info.find('div', class_='birthplace')
                      if birthplace_div:
                          birthplace_raw = birthplace_div.get_text(separator=", ").strip()
                          birthplace = ', '.join([part.strip() for part in birthplace_raw.split(',') if part.strip()])
                          birthplace_anchor = birthplace_div.find('a')
                          if birthplace_anchor:
                              birthplace_link = 'https://en.wikipedia.org' + birthplace_anchor.get('href')

              if 'Institutions' in header_text or 'Institution' in header_text:
                  institution_info = row.find('td')
                  if institution_info:
                      for a in institution_info.find_all('a'):
                          institutions.append(a.get_text(strip=True))
                          institutions_links.append('https://en.wikipedia.org' + a.get('href'))

      # Extract the photo link directly from the infobox
      # Look for img tags within the infobox
      img_tags = infobox.find_all('img', limit=1)
      if img_tags:
          img_src = img_tags[0].get('src')
          if img_src:
              photo_link = 'https:' + img_src
              # Validate if the link ends with image extension
              if not (photo_link.endswith('.jpg') or photo_link.endswith('.jpeg') or photo_link.endswith('.png')):
                  photo_link = None

      return birthplace, birthplace_link, institutions, institutions_links, photo_link

  # Add new columns for birthplace, birthplace_link, institutions, institutions_links, and photo_link
  df['birthplace'] = None
  df['birthplace_link'] = None
  df['institutions'] = None
  df['institutions_links'] = None
  df['photo_link'] = None

  # Iterate over the dataframe and fetch details
  for index, row in df.iterrows():
      #print(row['Name_Link'])
      birthplace, birthplace_link, institutions, institutions_links, photo_link = get_infobox_details(row['Name_Link'])
      df.at[index, 'birthplace'] = birthplace
      df.at[index, 'birthplace_link'] = birthplace_link
      df.at[index, 'institutions'] = institutions
      df.at[index, 'institutions_links'] = institutions_links
      df.at[index, 'photo_link'] = photo_link

      # Add a delay of 0.4 seconds between requests to comply with Wikipedia guidelines
      time.sleep(0.3)


  # Pickle the DataFrame
  pickle.dump([category,df], open(pklpath+'/'+ category+'df.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  [category,df] = pickle.load(open(pklpath+'/'+ category+'df.pkl', 'rb'))

print(df.head(3))

               Name                                       Name_Link  \
0   Vitaly Abalakov   https://en.wikipedia.org/wiki/Vitaly_Abalakov   
1   Ernst Karl Abbe        https://en.wikipedia.org/wiki/Ernst_Abbe   
2  Hovannes Adamian  https://en.wikipedia.org/wiki/Hovannes_Adamian   

                                          birthplace  \
0  Krasnoyarsk, Yeniseysk Governorate, Russian Em...   
1                     Eisenach, Saxe-Weimar-Eisenach   
2                               Baku, Russian Empire   

                             birthplace_link          institutions  \
0  https://en.wikipedia.org/wiki/Krasnoyarsk                    []   
1     https://en.wikipedia.org/wiki/Eisenach  [University of Jena]   
2         https://en.wikipedia.org/wiki/Baku                    []   

                                  institutions_links  \
0                                                 []   
1  [https://en.wikipedia.org/wiki/University_of_J...   
2                                        

In [25]:
print(df.columns)

Index(['Name', 'Name_Link', 'birthplace', 'birthplace_link', 'institutions',
       'institutions_links', 'photo_link'],
      dtype='object')


In [27]:
# get summary

import pandas as pd
import requests
import time


if True:

  # Function to fetch Wikipedia page summary
  def fetch_wikipedia_summary(link):
      time.sleep(0.2)  # Add a delay of 0.2 seconds between requests
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
          response = requests.get(url)
          data = response.json()
          if 'extract' in data:
              return data['extract']
      except Exception as e:
          print(f"Error fetching summary for {link}: {e}")
      return None

  # Apply the function to each row in the DataFrame
  df['summary'] = df['Name_Link'].apply(fetch_wikipedia_summary)


  # Pickle the DataFrame
  pickle.dump([category,df], open(pklpath+'/'+ category+'df.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  [category,df] = pickle.load(open(pklpath+'/'+ category+'df.pkl', 'rb'))


# Print the DataFrame with the added 'summary' column
print(df.head(2))

              Name                                      Name_Link  \
0  Vitaly Abalakov  https://en.wikipedia.org/wiki/Vitaly_Abalakov   
1  Ernst Karl Abbe       https://en.wikipedia.org/wiki/Ernst_Abbe   

                                          birthplace  \
0  Krasnoyarsk, Yeniseysk Governorate, Russian Em...   
1                     Eisenach, Saxe-Weimar-Eisenach   

                             birthplace_link          institutions  \
0  https://en.wikipedia.org/wiki/Krasnoyarsk                    []   
1     https://en.wikipedia.org/wiki/Eisenach  [University of Jena]   

                                  institutions_links  \
0                                                 []   
1  [https://en.wikipedia.org/wiki/University_of_J...   

                                          photo_link  \
0  https://upload.wikimedia.org/wikipedia/commons...   
1  https://upload.wikimedia.org/wikipedia/commons...   

                                             summary  
0  Vitaly Mikhaylov

In [29]:

# get coordinates

import pandas as pd
import requests
import time
import pickle


if True:


  # Function to fetch coordinates from Wikipedia API with error handling
  def fetch_coordinates_from_wikipedia(link):
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/w/api.php?action=query&prop=coordinates&format=json&titles={title}"
          response = requests.get(url)
          data = response.json()
          pages = data.get('query', {}).get('pages', {})
          if pages:
              page = next(iter(pages.values()))  # Get the first page (should be only one)
              coords = page.get('coordinates', [{}])[0]
              if coords:
                  return (coords.get('lat'), coords.get('lon'))
          # If there are no pages or coordinates, return None
          return None
      except Exception as e:
          #print(f"Error fetching coordinates for {link}: {e}")
          return None

  # Apply the function to each row in the DataFrame with a delay
  def fetch_coordinates_with_delay(links):
      coordinates = []
      for link in links:
          coordinates.append(fetch_coordinates_from_wikipedia(link))
          time.sleep(0.2)  # Add a delay of 0.2 seconds between requests
      return coordinates

  df['birth_coords'] = df['birthplace_link'].apply(fetch_coordinates_from_wikipedia)
  df['institutions_coords'] = df['institutions_links'].apply(fetch_coordinates_with_delay)


  # Pickle the DataFrame
  pickle.dump([category,df], open(pklpath+'/'+ category+'df.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  [category,df_large] = pickle.load(open(pklpath+'/'+ category+'df.pkl', 'rb'))

print(df.head(3))



               Name                                       Name_Link  \
0   Vitaly Abalakov   https://en.wikipedia.org/wiki/Vitaly_Abalakov   
1   Ernst Karl Abbe        https://en.wikipedia.org/wiki/Ernst_Abbe   
2  Hovannes Adamian  https://en.wikipedia.org/wiki/Hovannes_Adamian   

                                          birthplace  \
0  Krasnoyarsk, Yeniseysk Governorate, Russian Em...   
1                     Eisenach, Saxe-Weimar-Eisenach   
2                               Baku, Russian Empire   

                             birthplace_link          institutions  \
0  https://en.wikipedia.org/wiki/Krasnoyarsk                    []   
1     https://en.wikipedia.org/wiki/Eisenach  [University of Jena]   
2         https://en.wikipedia.org/wiki/Baku                    []   

                                  institutions_links  \
0                                                 []   
1  [https://en.wikipedia.org/wiki/University_of_J...   
2                                        

In [33]:

# grab all the categories in one df_all
# forgetting about cs people

import pickle

df_all = {}

categories = ['inventors','nobels']
for category in categories:
  # Load the pickled DataFrame
  [cat,df_all[category]] = pickle.load(open(pklpath+'/'+ category+'df.pkl', 'rb'))
  print(df_all[category].columns)


#print(df_all['nobels'][['Name','Field']])

Index(['Name', 'Name_Link', 'birthplace', 'birthplace_link', 'institutions',
       'institutions_links', 'photo_link', 'summary', 'birth_coords',
       'institutions_coords'],
      dtype='object')
Index(['Name', 'Name_Link', 'Field', 'Year', 'birthplace', 'birthplace_link',
       'institutions', 'institutions_links', 'photo_link', 'summary',
       'birth_coords', 'institutions_coords'],
      dtype='object')


In [35]:
# Add the Google Tag Manager and AdSense scripts to the map
gtag_script = """
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-2RB0G9NLE8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-2RB0G9NLE8');
</script>

<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-4771147286272001"
     crossorigin="anonymous"></script>
"""

In [43]:
import folium
from folium.plugins import MarkerCluster

# Initialize map centered on USA
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)


# Iterate through categories and add markers
for category, df in df_all.items():

    marker_cluster = MarkerCluster(name=category)  # Create MarkerCluster for each category
    
    # Example using birth_coords
    if 'birth_coords' in df.columns:
        for index, row in df.iterrows():
            coords = row['birth_coords']

            # Check if coords is a tuple and has exactly two elements
            if isinstance(coords, tuple) and len(coords) == 2:
                lat, lng = coords

                # Validate coordinates
                if isinstance(lat, (int, float)) and isinstance(lng, (int, float)):
                    # Determine icon based on category
                    if 'inventor' in category.lower():
                        formatted_category = f"<b>Inventor</b>"
                        icon = folium.Icon(icon="lightbulb", prefix="fa", icon_color="white", color="beige")
                    elif 'nobel' in category.lower():
                        formatted_category = f"<b>Nobel Prize Winner</b>"
                        icon = folium.Icon(icon="microscope", prefix="fa", icon_color="black", color="lightgray")
                    else:
                        formatted_category = f"<b>{category.capitalize()}</b>"
                        icon = folium.Icon(color="blue", icon="info-sign")

                    # Format popup content with wider width
                    popup_content = f"""
                    <div style="width: 300px;">
                        {formatted_category}<br>
                        <img src='{row['photo_link']}' width='100px'><br>
                        <b><big>{row['Name']}</big></b><br><br>
                        Born: {row['birthplace']}<br><br>
                        {row['summary']}<br><br>
                        <a href='{row['Name_Link']}' target='_blank'>Wikipedia Link</a>
                    </div>
                    """

                    folium.Marker([lat, lng], popup=popup_content, icon=icon).add_to(marker_cluster)
            else:
                print("*", end='')

    marker_cluster.add_to(m)  # Add MarkerCluster to the map

# Example Google Analytics tracking code
analytics = """
<!-- Google Analytics tracking code -->
<script>
  // Your Google Analytics JavaScript code here
</script>
"""

# Inject the analytics code into the map's HTML
m.get_root().html.add_child(folium.Element(analytics))

# Save the map as HTML file
path = 'maps'
map_filename = 'mapdiscoverers.html'
m.save(f"{path}/{map_filename}")

# Display the map (if in a Jupyter notebook or similar environment)
m


**************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************